# Experiment: presidential elections Ad

Data Collection is (**TODO** explain).  
It is somehow expensive (**TODO** explain).
You can communicate how (**TODO** explain). 

In this tutorial we are going to explain: a), b) , c)

# Table of Contents
* [Experiment: presidential elections Ads](#Experiment: presidential elections Ads)
* [Data collection: prepare data for your experiment](#Data-collection:-prepare-data-for-your-experiment)
    * [Set Up Dependencies](#Paragraph-3)
        * [Dep 1](#Paragraph-3)
        * [Dep 2](#Paragraph-3)
    * [Use pytube to download videos from you tube](#Paragraph-3)
        * [Task 1](#Paragraph-3)
        * [Task 2](#Paragraph-3)
    * [Use pydub to chunck videos and convert to audio](#Paragraph-3)
        * [Task 1](#Paragraph-3)
        * [Task 2](#Paragraph-3)
* [Feature Engineering: extract data using Azure public cloud](#Paragraph-3)
    * [Extract speech form audio](#Paragraph-3)
    * [Analyze thumbnails](#Paragraph-3)
    * [Key Phrases from speech](#Paragraph-3)
    * [Sentiment analysis](#Paragraph-3)
* [Modeling: Classification](#Paragraph-3)
    * [Presidential Elections commercials](#Paragraph-3)

## Data Collection: prepare data for your experiment

We are going to download videos from the webpage the [Ten of the Most Succesfull Presidential Campaign Ads Ever Made](https://www.kqed.org/lowdown/3955/ten-of-the-best-presidential-campaign-commercials-of-all-time). 

In [1]:
#import libraries
import os
import pandas as pd
from pytube import YouTube
import requests
import time

In [2]:
#list of URLs in the page (consider writing a script to scrape for greater quantities)
videos_urls = ['https://youtu.be/Y9RAxAgksSE', 
                'https://youtu.be/vs5ORK8RLWk', 
                'https://youtu.be/dDTBnsqxZ3k',
                'https://youtu.be/Qwk_epMblW4', 
                'https://youtu.be/qVcFUIXEDZ8', 
                'https://youtu.be/EU-IBF8nwSY',
                'https://youtu.be/PmwhdDv8VrM', 
                'https://youtu.be/Xq_x3JUwrU0', 
                'https://youtu.be/pbdzMLk9wHQ', 
                'https://youtu.be/jjXyqcx-mYY']

In [3]:
#set directory to the folder where we are going to download the videos
os.chdir('../../data/video/video.mp4/')
dir_video = os.getcwd()

#set directory to folder to download the video thumbnail
os.chdir('../thumbnail/')
dir_thumbnail = os.getcwd()

#print directories where files are going to be saved
print(dir_video)
print(dir_thumbnail)

C:\Users\popor\iqss_workshop\workshops\public_cloud_computing\data\video\video.mp4
C:\Users\popor\iqss_workshop\workshops\public_cloud_computing\data\video\thumbnail


In [24]:
#set procedure starting time
print('-------------------')
print("Start downloading videos and thumbnails")
print('-------------------')
start = time.time()

#store useful info to lists
titles = []
thumbnail_urls = []
video_file_spec = []

#download videos and thumbnails
for video in videos_urls:

    #instatiate Youtube object passing video URL
    yt = YouTube(video)

    #extract the video's title and thumbnail url
    replacements = {" ": "_", "-": "", "'": "", '"': "", ':':'', '.':'', '(':'', ')':''}
    title = "".join([replacements.get(c, c) for c in yt.title.lower()])
    titles.append(title)
    thumbnail_url = yt.thumbnail_url.lower()
    thumbnail_urls.append(thumbnail_url)

    #choose file to download (filter by extension and resolution and take first in the list)
    video_file = yt.streams.filter(file_extension = 'mp4', res="360p").first()
    video_file_spec.append(video_file)
    
    #download video
    try:
        video_file.download(dir_video, filename=title)
        print('{} download status: success'.format(title))
    except:
        print('{} download status: fail'.format(title))

    #set thumbail name and choose path to save it
    thumbnail_name = title +'_'+ yt.thumbnail_url.split('/')[-1]
    save_thumbnail_path = (os.path.join(dir_thumbnail, thumbnail_name))
    
    #download thumbnail
    img_data = requests.get(thumbnail_url).content
    with open(save_thumbnail_path, 'wb') as handler:
        try:
            handler.write(img_data)
            print('{} download status: success'.format(thumbnail_name))
        except:
            print('{} download status: fail'.format(thumbnail_name))

#set procedure ending time
end = time.time()
print('-------------------')
print('Download completed')
print('-------------------')
print('It took {} seconds to download 10 Ads and their thumbnails'.format(round(end - start),2))

-------------------
Start downloading videos and thumbnails
-------------------
eisenhower_for_president_1952 download status: success
eisenhower_for_president_1952_default.jpg download status: success
kennedy_for_me_campaign_jingle_jfk_1960 download status: success
kennedy_for_me_campaign_jingle_jfk_1960_default.jpg download status: success
high_quality_famous_daisy_attack_ad_from_1964_presidential_election download status: success
high_quality_famous_daisy_attack_ad_from_1964_presidential_election_default.jpg download status: success
humphrey_laughing_at_spiro_agnew_1968_political_ad download status: success
humphrey_laughing_at_spiro_agnew_1968_political_ad_default.jpg download status: success
mcgovern_defense_plan_ad_nixon_1972_presidential_campaign_commercial download status: success
mcgovern_defense_plan_ad_nixon_1972_presidential_campaign_commercial_default.jpg download status: success
ronald_reagan_tv_ad_its_morning_in_america_again download status: success
ronald_reagan_tv_ad_

In [20]:
#import libraries
import pydub
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import make_chunks

In [21]:
#set path to folder to save audio file
os.chdir('../audio/')
dir_audio = os.getcwd()

print(dir_audio)

C:\Users\popor\iqss_workshop\workshops\public_cloud_computing\data\video\audio


In [22]:
#set procedure starting time
print('-------------------')
print("Start dividing video in chuncks and converting to audio")
print('-------------------')
start = time.time()

#store audio lenght
audio_lenght = []

#convert video to audio, and split in chunks
for title in titles:
    
    #set path to file to convert
    path_to_file = os.path.join(dir_video, title) + '.mp4'
        
    #read file into variable
    myaudio = AudioSegment.from_file(path_to_file, "mp4")
    
    #estimate length
    file_length = round(len(myaudio)/1000, 2)
    audio_lenght.append(file_length)
    
    #Make chunks of fi
    chunk_length_ms = 10000
    chunks = make_chunks(myaudio, chunk_length_ms)

    #take each chunck and export it to path as wav
    dir_file = os.path.join(dir_audio, title)
    for i, chunk in enumerate(chunks):
        chunk.export(dir_file + "_chunck_{}.wav".format(i+1), format="wav")
        
    print('The video {0} is long {1} seconds and has been split in {2} audio files'.format(title, file_length,len(chunks)))
    
#set procedure ending time
end = time.time()
print('-------------------')
print('Process completed')
print('-------------------')
print('It took {} seconds to chunk the video and convert '.format(round(end - start, 2)))

-------------------
Start dividing video in chuncks and converting to audio
-------------------
The video eisenhower_for_president_1952 is long 62.09 seconds and has been split in 7 audio files
The video kennedy_for_me_campaign_jingle_jfk_1960 is long 60.23 seconds and has been split in 7 audio files
The video high_quality_famous_daisy_attack_ad_from_1964_presidential_election is long 66.9 seconds and has been split in 7 audio files
The video humphrey_laughing_at_spiro_agnew_1968_political_ad is long 19.25 seconds and has been split in 2 audio files
The video mcgovern_defense_plan_ad_nixon_1972_presidential_campaign_commercial is long 60.05 seconds and has been split in 7 audio files
The video ronald_reagan_tv_ad_its_morning_in_america_again is long 59.95 seconds and has been split in 6 audio files
The video 1988_george_bush_sr_revolving_door_attack_ad_campaign is long 29.88 seconds and has been split in 3 audio files
The video bill_clinton_hope_ad_1992 is long 60.26 seconds and has be

In [ ]:
#TODO split on silence

#set procedure starting time
print('-------------------')
print("Start dividing video in chuncks and converting to audio")
print('-------------------')
start = time.time()

#store audio lenght
audio_lenght = []

#convert video to audio, and plit in chunks
for title in titles:

    #set path to file to convert
    path_to_file = os.path.join(dir_video, title) + '.mp4'
        
    #read file 
    sound = AudioSegment.from_file(path_to_file, "mp4")
    
    #estimate length
    file_length = round(len(sound)/1000, 2)
    audio_lenght.append(file_length)
    

    print(title)
    
    #split audio when it is silent for at least one second or quieter than -16 dBFS
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=-16,  keep_silence=200)
    print(len(chunks))
    
    if len(chunks) == 0:
        chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=-48,  keep_silence=200)
    print(len(chunks))
    
    if len(chunks) == 1:
        chunks = split_on_silence(sound, min_silence_len=300, silence_thresh=-72,  keep_silence=200)
    print(len(chunks))
    
    # now recombine the chunks so that the parts are at least 10 sec long
    target_length = 10 * 1000
    output_chunks = [chunks[0]]
    for chunk in chunks[1:]:
        if len(output_chunks[-1]) < target_length:
            output_chunks[-1] += chunk
        else:
            # if the last output chunk is longer than the target length start a new one
            output_chunks.append(chunk)
    
    print(file_length,len(chunks))
    
    dir_file = os.path.join(dir_audio, title)
    #take each chunck and save it as wav
    for i, chunk in enumerate(output_chunks):
        chunk.export(dir_file + "_chunck_{}.wav".format(i+1), format="wav")
        
    print('The video {0} is long {1} seconds and has been split in {2} audio files'.format(title, file_length,len(output_chunks)))
    
#set procedure ending time
end = time.time()
print('-------------------')
print('Process completed')
print('-------------------')
print('It took {} seconds to chunk the video and convert '.format(round(end - start, 2)))

In [ ]:
#sample code to check split on silence

title_HS = titles[4]
path_to_file = os.path.join(dir_video, title_HS) + '.mp4'
sound = AudioSegment.from_file(path_to_file, "mp4")
file_length = round(len(sound)/1000, 2)

chunks = split_on_silence(sound, min_silence_len=300, silence_thresh=-16,  keep_silence=200) #,  keep_silence=200
print(title_HS, len(chunks))

if len(chunks) == 0:
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=-32,  keep_silence=200)
print(len(chunks))

if len(chunks) == 0:
    chunks = split_on_silence(sound, min_silence_len=300, silence_thresh=-32,  keep_silence=200)
print(len(chunks))

target_length =  10 * 1000
    
output_chunks = [chunks[0]]
for chunk in chunks[1:]:
    if len(output_chunks[-1]) < target_length:
        output_chunks[-1] += chunk
    else:
        # if the last output chunk is longer than the target length start a new one
        output_chunks.append(chunk)
        
print(file_length,len(chunks))

dir_file = os.path.join(dir_audio, title_HS)
#take each chunck and save it as wav
for i, chunk in enumerate(output_chunks):
    chunk.export(dir_file + "_chunck_{}.wav".format(i+1), format="wav")

print('The video {0} is long {1} seconds and has been split in {2} audio files'.format(title, file_length,len(output_chunks)))

| title | min_silence_len| silence_thresh |keep_silence| target_length |
|---|---|---|---|---|
| eisenhower | 500 | -16 | 200 | 10 |
| kennedy | 500 | -16 | 200 | 10 |
| LBJ | 500 | -32 | 200 | 5 |
| HHH | 500 | -16 | 200 | 5 |



In [ ]:
#add some predictors
year = ['1952','1960','1964','1968','1972','1984','1988','1992','2004','2008']
candidate = ['IKE', 'JFK', 'LBJ', 'HHH', 'NIX','---','---','---','---','---' ]
party = ['republican','democratic','democratic','republican', 'republican','---','---','---','---','---',]
win = ['-','-','-',0,'-','-','-','-','-','-']

df = pd.DataFrame({'video_url':videos_urls,
                   'thumbnail_url':thumbnail_urls,
                   'title':titles,
                   'length[sec]':audio_lenght,
                   'year':year,
                   'candidate_name':candidate,
                   'party': party,
                   'win': win})

df

In [ ]:
#dump the dataframe on a file
import pickle
import json

#dump the dataframe on a file
df.to_pickle('../Ads_presidential_election')

In [25]:
#load dataframe on a file
df = pd.read_pickle('../Ads_presidential_election')
df

,video_url,thumbnail_url,titles,length[sec],year,candidate_name,party,win
0,https://youtu.be/Y9RAxAgksSE,https://i.ytimg.com/vi/y9raxagksse/default.jpg,eisenhower_for_president_1952,62.09,1952,IKE,republican,-
1,https://youtu.be/vs5ORK8RLWk,https://i.ytimg.com/vi/vs5ork8rlwk/default.jpg,kennedy_for_me_campaign_jingle_jfk_1960,60.23,1960,JFK,democratic,-
2,https://youtu.be/dDTBnsqxZ3k,https://i.ytimg.com/vi/ddtbnsqxz3k/default.jpg,high_quality_famous_daisy_attack_ad_from_1964_...,66.90,1964,LBJ,democratic,-
3,https://youtu.be/Qwk_epMblW4,https://i.ytimg.com/vi/qwk_epmblw4/default.jpg,humphrey_laughing_at_spiro_agnew_1968_politica...,19.25,1968,HHH,republican,0
4,https://youtu.be/qVcFUIXEDZ8,https://i.ytimg.com/vi/qvcfuixedz8/default.jpg,mcgovern_defense_plan_ad_nixon_1972_presidenti...,60.05,1972,NIX,republican,-
5,https://youtu.be/EU-IBF8nwSY,https://i.ytimg.com/vi/eu-ibf8nwsy/default.jpg,ronald_reagan_tv_ad_its_morning_in_america_again,59.95,1984,---,---,-
6,https://youtu.be/PmwhdDv8VrM,https://i.ytimg.com/vi/pmwhddv8vrm/default.jpg,1988_george_bush_sr_revolving_door_attack_ad_c...,29.88,1988,---,---,-
7,https://youtu.be/Xq_x3JUwrU0,https://i.ytimg.com/vi/xq_x3juwru0/default.jpg,bill_clinton_hope_ad_1992,60.26,1992,---,---,-
8,https://youtu.be/pbdzMLk9wHQ,https://i.ytimg.com/vi/pbdzmlk9whq/default.jpg,historical_campaign_ad_windsurfing_bushcheney_04,30.09,2004,---,---,-
9,https://youtu.be/jjXyqcx-mYY,https://i.ytimg.com/vi/jjxyqcx-myy/default.jpg,yes_we_can__barack_obama_music_video,270.21,2008,---,---,-


In [ ]:
#TODO

#other features
###number/percentage of voters for candidate for presidential elections
###word count

#advanced features
###extract frames from video (https://stackoverflow.com/questions/27481993/extracting-image-from-video-at-a-given-time-using-opencv)
###number of views
###users comments on you tube

In [ ]:
#store file into a blob container

#detect face in thumbnails
#analyze image
#character recognition

#speech recognition (basic words count / bag of words)
#key phrases
#sentiment analyzes

#assemble into a dataframe

In [ ]:
[The Living Room Candidate at Museum of the Moving Image: presidential campaign commercials](http://www.livingroomcandidate.org/)